# Introduction

This notebook aims to explore trends in recent data on NYC motor vehicle collisions. Cleaned data (from data_exploration/Obtaining_and_Cleaning_the_Data.ipynb notebook) will be joined (inner join) to the Vehicle Collisions - Crashes Joined to Neighborhood Data.csv (Kevin Lee's csv with neighborhood name column).

### Tasks
1. Visualize data on the borough level (borough boundaries)
2. Visualize data on the neigborhood level (neighborhood boundaries)
3. Visualize data using H3
4. Create Hexagon boundaries within neighborhoods

#### Use plotly for vis


# Data Source

The data used in this notebook was obtained from: 

- [NYC Open Data's Motor Vehicle Collision-Crashes](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95)
  - This dataset contains information from all police reported motor vehicle collisions in NYC. Each row represents a crash event.The police report (MV104-AN) is required to be filled out for collisions where someone is injured or killed, or where there is at least 1000 dollars worth of damage. This notebook uses a subset of the data and was accessed with the [Socrata Open Data (SODA) API](https://dev.socrata.com/consumers/getting-started.html). 
 

# Loading Dependencies

In [1]:
import json
import requests
import pandas as pd
#from sodapy import Socrata
import numpy as np
import urllib.request
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'
#from urllib.request import urlopen
import geopandas as gdp
from geojson import Feature, Point, FeatureCollection, Polygon

/Users/jorgegranda/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



Create access token (set of permissions that allow the token to make certain types of requests to Mapbox APIs)

In [2]:
px.set_mapbox_access_token(open(".mapbox_token").read())

In [ ]:
# “I suggested to use edit distance + agglomerative clustering + spell checking to cleaning up mispelled categories”.

**Borough + Neighborhoods Geojson files**

In [3]:
# Obtain geojson file with neighborhood boundaries
neigh_url = 'https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Neighborhood_Tabulation_Areas_2020/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson'

In [4]:
#boro_url = 'https://services1.arcgis.com/0mhlGDIUKwQF6tnJ/arcgis/rest/services/NYC_Borough_Boundaries/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson'

In [5]:
# boro_json = requests.get(boro_url)
# boro_json = boro_json.json()

In [39]:
#boro_json['features'][0]['properties']

In [4]:
hood_json = requests.get(neigh_url)
hood_json = hood_json.json()

In [5]:
hood_json['features'][0]['properties']

{'OBJECTID': 1,
 'BoroCode': 3,
 'BoroName': 'Brooklyn',
 'CountyFIPS': '047',
 'NTA2020': 'BK0101',
 'NTAName': 'Greenpoint',
 'NTAAbbrev': 'Grnpt',
 'NTAType': '0',
 'CDTA2020': 'BK01',
 'CDTAName': 'BK01 Williamsburg-Greenpoint (CD 1 Equivalent)',
 'Shape__Area': 35321930.6625214,
 'Shape__Length': 28914.3147173552}

In [6]:
# Obtain geojson file with Borough boundaries
nyc_boro = json.load(open("Data/Borough Boundaries.geojson", "r"))

In [43]:
nyc_boro['features'][0]['properties']

{'boro_code': '4',
 'boro_name': 'Queens',
 'shape_area': '3040205594.95',
 'shape_leng': '900269.280485'}

# Importing the Data


In [13]:
#df_crash_count = pd.read_csv('Data/Borough and Neighborhood Crash Counts (2012-07-01 through 2022-03-15).csv')
# df_motor_vehicle = pd.read_csv('Data/Motor Vehicle Collisions - Crashes Joined to Neighborhood Data.csv')
# df_original = pd.read_csv('Data/NYC-Open-Data-Motor-Vehicle-Collision-Crashes.csv')#
df_motor_vehicle = pd.read_csv('Data/Motor-Vehicle-Collision-Crashes_12M.csv')

In [14]:
df_motor_vehicle.shape

(96580, 12)

#### 1. Visualize data on the borough level (borough boundaries)

    - Create choropleth map of collisions and number of persons injured per NYC borough

In [15]:
# Use the same group by structure for neigborhoods and boroughs
df_gb_boro = df_motor_vehicle.groupby('boroname').agg(
            number_of_collisions = ('collision_id', 'count'),
            number_of_persons_injured = ('number_of_persons_injured', sum)).reset_index()

In [16]:
df_gb_boro.head()

,boroname,number_of_collisions,number_of_persons_injured
0,Bronx,17386,8348
1,Brooklyn,32581,15846
2,Manhattan,15831,6917
3,Queens,26252,12519
4,Staten Island,4530,2011


#### Borough Map

In [19]:
boro_fig = px.choropleth_mapbox(
    df_gb_boro,
    locations = "boroname",
    geojson = nyc_boro,
    color = "number_of_persons_injured",
    featureidkey="properties.boro_name",
    #color_continuous_scale=px.colors.continuous.Viridis[::-1],
    color_continuous_scale=px.colors.sequential.YlOrRd,
    #color_continuous_scale="viridis",
    #px.colors.sequential.Viridis,
    hover_name="boroname",
    hover_data={'number_of_persons_injured': True,
                'number_of_collisions': True,
                'boroname': False},
    mapbox_style="carto-positron",
    center={"lat": 40.730610, "lon": -73.9749},
    zoom=8.5,
    opacity=0.5,
    title = "NYC Boroughs",)
# fig.update_layout(
#     title={
#         'text': "location of fatalities",
#         'y':0.9,
#         'x':0.5,
#         'xanchor': 'center',
#         'yanchor': 'top'})

In [20]:
boro_fig.show()

#### 2. Visualize data on the neigborhood level (neighborhood boundaries)
#### Create choropleth map of collisions and number of persons injured per NYC neigborhood
- Use similar "group by" structure for data sumary on neighborhoods and boroughs
- Group by locations (Boro, Neighborhood)
    - Collision id column agg 'count' (will count the number of collitions at each location)
    - Persons injured column agg 'sum' (will add the number of persons injured at each location)

#### Group dataframe by borough/neighborhood + aggregations
**Filtering**
- Selecting Borough
    - Displaying neighborhoods from selected borough

In [17]:
df_gb_boro_neigh = df_motor_vehicle.groupby(['boroname','ntaname']).agg(
            number_of_collisions = ('collision_id', 'count'),
            number_of_persons_injured = ('number_of_persons_injured', sum)).reset_index()

In [18]:
df_gb_boro_neigh.head()

,boroname,ntaname,number_of_collisions,number_of_persons_injured
0,Bronx,Allerton,208,139
1,Bronx,Bedford Park,325,101
2,Bronx,Belmont,488,240
3,Bronx,Bronx Park,266,153
4,Bronx,Castle Hill-Unionport,522,250


In [19]:
df_gb_boro_neigh['injuries_per_collision'] = round((df_gb_boro_neigh['number_of_persons_injured']/df_gb_boro_neigh['number_of_collisions']),2)

In [20]:
df_gb_boro_neigh.head()

,boroname,ntaname,number_of_collisions,number_of_persons_injured,injuries_per_collision
0,Bronx,Allerton,208,139,0.67
1,Bronx,Bedford Park,325,101,0.31
2,Bronx,Belmont,488,240,0.49
3,Bronx,Bronx Park,266,153,0.58
4,Bronx,Castle Hill-Unionport,522,250,0.48


**Neighboorhood Map**

In [24]:
neighborhood_fig = px.choropleth_mapbox(
    df_gb_boro_neigh,
    locations = "ntaname",
    geojson = hood_json,
    color = "number_of_persons_injured",
    featureidkey="properties.NTAName",
    #color_continuous_scale=px.colors.continuous.Viridis[::-1],
    #colorscale = 'Reds',
    color_continuous_scale=px.colors.sequential.YlOrRd,
    #color_continuous_scale="viridis",
    #px.colors.sequential.Viridis,
    hover_name="ntaname",
    hover_data={'number_of_persons_injured': True,
                'number_of_collisions': True,
                'injuries_per_collision': True,
                #'boroname': False,
                'ntaname': False},
    mapbox_style="carto-positron",
    center={"lat": 40.730610, "lon": -73.9749},
    zoom=10,
    opacity=0.5,
    title = "Neighboorhood Map")
# fig.update_layout(
#     title={
#         'text': "location of fatalities",
#         'y':0.9,
#         'x':0.5,
#         'xanchor': 'center',
#         'yanchor': 'top'})

In [25]:
neighborhood_fig.show()

In [26]:
# Filtering Brooklyn data
borough = 'Brooklyn'
df_one_boro = df_gb_boro_neigh[df_gb_boro_neigh['boroname'] == borough]

In [ ]:
#df_one_boro = df_gb_boro_neigh.loc[(df_gb_boro_neigh['boroname'] == Boro_name) & (df_gb_boro_neigh['number_of_collisions']> 100)]


#### Map of Neighborhoods from selected Borough

In [30]:
one_boro_fig = px.choropleth_mapbox(
    df_one_boro,
    locations = "ntaname",
    geojson = hood_json,
    color = "number_of_persons_injured",
    featureidkey="properties.NTAName",
    #color_continuous_scale=px.colors.continuous.Viridis[::-1],
    #colorscale = 'Reds',
    color_continuous_scale=px.colors.sequential.YlOrRd,
    #color_continuous_scale="viridis",
    #px.colors.sequential.Viridis,
    hover_name="ntaname",
    hover_data={'number_of_persons_injured': True,
                'number_of_collisions': True,
                'injuries_per_collision': True,
                #'boroname': False,
                'ntaname': False},
    mapbox_style="carto-positron",
    center={"lat": 40.730610, "lon": -73.9749},
    zoom=10,
    opacity=0.5,
    title = "Map of : " + borough,)

In [31]:
one_boro_fig.show()

# Visualizing Collisions with H3  
[H3 Documentation](https://h3geo.org/docs/)

In [33]:
import h3
from shapely.geometry import Polygon     

### Create data frame that includes collision_id, borough name, neighborhood name, lat, lon and persons injured columns
- Lat and lon will be used to create he3 cells and to populate cells with corresponding data
- Collision id for counting the total number of collisions per location (h3 cell)
- Number of persons injured (sum total per corresponding location)
- Borough name column will allow data filtering by borough
- Neighborhood column will allow data filtering by neigborhood name



In [63]:
df_motor_vehicle.columns

Index(['collision_id', 'crash date', 'crash time', 'latitude', 'longitude',
       'boroname', 'ntaname', 'cdtaname', 'geometry',
       'number_of_persons_killed', 'number_of_persons_injured', 'year'],
      dtype='object')

In [35]:

df_h3 = (df_motor_vehicle[['collision_id','boroname','ntaname','latitude','longitude','number_of_persons_injured']])

In [36]:
df_h3.head()

,collision_id,boroname,ntaname,latitude,longitude,number_of_persons_injured
0,4407147,Brooklyn,Park Slope,40.683580,-73.97617,1
1,4407702,Brooklyn,Park Slope,40.669067,-73.98780,0
2,4407489,Brooklyn,Park Slope,40.673008,-73.97851,0
3,4407699,Brooklyn,Park Slope,40.681335,-73.97667,1
4,4407890,Brooklyn,Park Slope,40.682842,-73.97660,1


- Function to map car collision points to the H3 cells.
- H3 resolution 9 (.1 square km hexagon) 

In [37]:
H3_res = 9 # H3 Resolution (.1km^2)
def geo_to_h3(row):
  return h3.geo_to_h3(lat=row.latitude,lng=row.longitude,resolution = H3_res)

In [38]:
df_h3['h3_cell'] = df_h3.apply(geo_to_h3,axis=1)

/Users/jorgegranda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [39]:
df_h3.shape

(96580, 7)

### Group dataframe by h3 column, boroname, ntaname and aggregates collition id (count), number of persons injured (sum)

In [40]:
# df_gb_boro_neigh = df_motor_vehicle.groupby(['boroname','ntaname']).agg(
#             number_of_collisions = ('collision_id', 'count'),
#             number_of_persons_injured = ('number_of_persons_injured', sum)).reset_index()
#df_h3_gb = (df_h3.groupby('h3_cell').number_of_persons_injured.sum()).reset_index()
## Grouping by h3 (counting ids, adding number of persons injured)
# df_h3_gb = df_h3.groupby('h3_cell').agg(
#         # Get count of id per cell
#         number_of_collisions = ('collision_id', "count"),
#         number_of_persons_injured = ('number_of_persons_injured', sum)).reset_index()


In [40]:
df_h3_gb = df_h3.groupby(['h3_cell','boroname','ntaname']).agg(
        # Get count of id per cell
        number_of_collisions = ('collision_id', "count"),
        number_of_persons_injured = ('number_of_persons_injured', sum)).reset_index()

In [41]:
df_h3_gb.shape

(7749, 5)

In [75]:
# One h3 cell may partly be included in more than one neighborhood
df_h3_gb.loc[df_h3_gb.h3_cell.str.contains('892a1076203ffff')] #892a100dea7ffff
#df_neiborhood = df_h3_gb3.loc[df_h3_gb3.ntaname.str.contains('astoria', case = False)]


,h3_cell,boroname,ntaname,number_of_collisions,number_of_persons_injured,geometry
6989,892a1076203ffff,Brooklyn,Flatbush,8,6,POLYGON ((-73.95342596865591 40.63367083801214...


Needed for plotting
- Use shapely (import Polygon)
- Function to generate hexagon geometry for each cell


In [42]:
# Function to generate hexagon geometry for each hexagon
def add_geometry(row):
  points = h3.h3_to_geo_boundary(row['h3_cell'], True)
  return Polygon(points)

In [43]:
df_h3_gb['geometry'] = (df_h3_gb.apply(add_geometry,axis=1))

In [44]:
df_h3_gb.head()

,h3_cell,boroname,ntaname,number_of_collisions,number_of_persons_injured,geometry
0,892a1000047ffff,Bronx,Pelham Bay-Country Club-City Island,1,0,POLYGON ((-73.79056100760786 40.85996287572233...
1,892a1000063ffff,Bronx,Pelham Bay-Country Club-City Island,6,1,POLYGON ((-73.79037556521729 40.85467378813006...
2,892a1000067ffff,Bronx,Pelham Bay-Country Club-City Island,8,4,POLYGON ((-73.78814034418616 40.85208351682743...
3,892a1000073ffff,Bronx,Pelham Bay-Country Club-City Island,1,0,POLYGON ((-73.78832552397652 40.85737243243801...
4,892a100007bffff,Bronx,Pelham Bay-Country Club-City Island,4,3,POLYGON ((-73.79261103722521 40.85726410455732...


### For the Choropleth map
- Data frame + Geojson-dictionary
    - Create a GeoJSON-formatted dictionary using Dataframe
    - Need a location column to assign color to map


In [45]:
def hexagons_dataframe_to_geojson(df_hex, hex_id_field,geometry_field, value_field,file_output = None):

    list_features = []

    for i, row in df_hex.iterrows():
        feature = Feature(geometry = row[geometry_field],
                          id = row[hex_id_field],
                          properties = {"value": row[value_field]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    if file_output is not None:
        with open(file_output, "w") as f:
            json.dump(feat_collection, f)

    else :
      return feat_collection

In [46]:
# GeoJson object
# Only one file needs to be created for H3 resolution
geojson_obj = (hexagons_dataframe_to_geojson
                       (df_h3_gb,
                        hex_id_field='h3_cell',
                        value_field='number_of_persons_injured',
                        geometry_field='geometry'))

In [71]:
df_h3_gb.columns

Index(['h3_cell', 'boroname', 'ntaname', 'number_of_collisions',
       'number_of_persons_injured', 'geometry'],
      dtype='object')

In [77]:
df_h3_gb.boroname.unique()

array(['Bronx', 'Queens', 'Manhattan', 'Brooklyn', 'Staten Island'],
      dtype=object)

### Filter df_h3_gb dataset to view data from one Borough**
- Further filter data from borough to view data from one neighboorhood

In [64]:
borough = 'Brooklyn'
df_boro = df_h3_gb[df_h3_gb['boroname'] == borough]

Figure of hexagones covering selected borough (data from one of the boroughs)


In [65]:
# Fig colored by number of persons injured
boro_hex_fig = (px.choropleth_mapbox(
                    df_boro, # Passing selected neighborhood
                    #df_h3_gb3, # Passing dataframe covering all neighborhoods
                    geojson=geojson_obj, 
                    locations='h3_cell',
                    #featureidkey = "features.NTAName",
                    color='number_of_persons_injured',
                    #color_continuous_scale=px.colors.continuous.Viridis[::-1],
                    #color_continuous_scale=px.colors.sequential.Inferno[::-1],
                    color_continuous_scale=px.colors.sequential.Inferno[::-1],
                    #color_continuous_scale="viridis",
                    mapbox_style='carto-positron',
                    zoom=8,
                    center = {"lat": 40.730610, "lon": -73.9749},
                    opacity=0.7,
                    title = "Number of Persons Injured at Location",
                    hover_name="ntaname",
                    #hover_data= ["number_of_persons_injured"], {'h3_cell':False}
                    hover_data={'h3_cell':False,
                               'number_of_persons_injured': True,
                               'number_of_collisions': True,
                               'ntaname': False}
))

In [66]:
boro_hex_fig.show()

**Filter df_boro to view data from selected neighborhood**

In [50]:
# Pick neighborhood from selected borough
neighborhood = 'Flatbush'
df_neighborhood = df_boro[df_boro['ntaname'] == neighborhood]

In [74]:
df_neighborhood.head()

,h3_cell,boroname,ntaname,number_of_collisions,number_of_persons_injured,geometry
6989,892a1076203ffff,Brooklyn,Flatbush,8,6,POLYGON ((-73.95342596865591 40.63367083801214...
6990,892a1076207ffff,Brooklyn,Flatbush,3,1,POLYGON ((-73.95119199394261 40.63109089320743...
6992,892a107620bffff,Brooklyn,Flatbush,3,1,"POLYGON ((-73.95769129184109 40.6335563373286,..."
6994,892a107620fffff,Brooklyn,Flatbush,2,0,POLYGON ((-73.95545708095921 40.63097647419458...
6997,892a1076213ffff,Brooklyn,Flatbush,37,21,POLYGON ((-73.95139462674628 40.63636525335176...


Figure of hexagones covering selected neighborhood


In [76]:
hex_neighborhood_fig = (px.choropleth_mapbox(
                    df_neighborhood, # Passing selected neighborhood
                    #df_h3_gb3, # Passing dataframe covering all neighborhoods
                    geojson=geojson_obj, 
                    locations='h3_cell',
                    #featureidkey = "features.NTAName", #feature key from neighboorhood geojson file
                    color='number_of_persons_injured',
                    #color_continuous_scale=px.colors.continuous.Viridis[::-1],
                    #color_continuous_scale=px.colors.sequential.Inferno[::-1],
                    #color_continuous_scale=px.colors.sequential.Inferno[::-1],
                    color_continuous_scale=px.colors.sequential.YlOrRd,
                    #color_continuous_scale="viridis",
                    mapbox_style='carto-positron',
                    zoom=12,
                    center = {"lat": 40.633670, "lon": -73.953425},
                    opacity=0.7,
                    title = "Number of Persons Injured at Location",
                    hover_name="ntaname",
                    #hover_data= ["number_of_persons_injured"], {'h3_cell':False}
                    hover_data={'h3_cell':False,
                               'number_of_persons_injured': True,
                               'number_of_collisions': True,
                               'ntaname': False}
))

In [77]:
hex_neighborhood_fig.show()

In [72]:
df_h3_gb.ntaname.unique()

array(['Pelham Bay-Country Club-City Island', 'Co-op City',
       'Pelham Bay Park', 'Eastchester-Edenwald-Baychester',
       'Fort Totten', 'Douglaston-Little Neck', 'Whitestone-Beechhurst',
       'Throgs Neck-Schuylerville', 'Bay Terrace-Clearview', 'Bayside',
       'Auburndale', 'Williamsbridge-Olinville', 'Pelham Gardens',
       'Allerton', 'Bronx Park', 'Norwood', 'Morris Park',
       'Hutchinson Metro Center', 'Westchester Square', 'Bedford Park',
       'Belmont', 'Kingsbridge Heights-Van Cortlandt Village',
       'University Heights (North)-Fordham', 'Van Cortlandt Park',
       'Pelham Parkway-Van Nest', 'West Farms', 'Tremont',
       'Wakefield-Woodlawn', 'Woodlawn Cemetery', 'Castle Hill-Unionport',
       'Ferry Point Park-St. Raymond Cemetery', 'Soundview-Clason Point',
       'College Point', 'Soundview Park',
       'Soundview-Bruckner-Bronx River', 'Hunts Point',
       'Crotona Park East', 'Parkchester',
       'Glen Oaks-Floral Park-New Hyde Park', 'Bellerose'

**Neighborhood data can also be selected directly from original df_h3_gb dataset**

In [60]:
# Pick a neighborhood in Brooklyn from df_h3_gb original data
neighborhood = 'Flatbush'
df_neighborhood = df_h3_gb[df_h3_gb['ntaname'] == neighborhood]

In [61]:
# Fig colored by number of persons
hex_neighborhood_fig2 = (px.choropleth_mapbox(
                    df_neighborhood, # Passing selected neighborhood
                    #df_h3_gb3, # Passing dataframe covering all neighborhoods
                    geojson=geojson_obj, 
                    locations='h3_cell',
                    #featureidkey = "features.NTAName",
                    color='number_of_persons_injured',
                    #color_continuous_scale=px.colors.continuous.Viridis[::-1],
                    #color_continuous_scale=px.colors.sequential.Inferno[::-1],
                    #color_continuous_scale=px.colors.sequential.Inferno[::-1],
                    color_continuous_scale=px.colors.sequential.YlOrRd,
                    mapbox_style='carto-positron',
                    zoom=7,
                    center = {"lat": 40.730610, "lon": -73.9749},
                    opacity=0.7,
                    title = "Number of Persons Injured at Location",
                    hover_name="ntaname",
                    #hover_data= ["number_of_persons_injured"], {'h3_cell':False}
                    hover_data={'h3_cell':False,
                               'number_of_persons_injured': True,
                               'number_of_collisions': True,
                               'ntaname': False}
))


In [62]:
hex_neighborhood_fig2.show()